In [ ]:
import os
import json
from flask import Flask, request, render_template_string
from openai import OpenAI

app = Flask(__name__)

# OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# -------------------------
# Calorie estimation
# -------------------------
def estimate_daily_calories(weight_kg, height_cm):
    bmr = 10 * weight_kg + 6.25 * height_cm - 100
    activity_factor = 1.6
    return int(bmr * activity_factor)

# -------------------------
# Ask ChatGPT for meals
# -------------------------
def get_ai_meal_recommendations(profile):
    prompt = f"""
You are a sports nutrition assistant for pickleball players.

Athlete profile:
Height: {profile['height']} cm
Weight: {profile['weight']} kg
Estimated daily calories: {profile['daily_cal']} kcal
Play duration: {profile['duration']}
Meal timing: {profile['timing']}

Recommend 3 appropriate meals for this timing window.
Each meal should include:
- name
- estimated calories
- short description

Respond ONLY in valid JSON in this format:

{{
  "meals": [
    {{"name": "...", "calories": 400, "description": "..."}}
  ]
}}
"""

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You are a helpful sports nutrition assistant."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.7,
    )

    content = response.choices[0].message.content

    try:
        data = json.loads(content)
        return data["meals"]
    except Exception as e:
        print("JSON parse error:", e)
        print(content)
        return []

# -------------------------
# HTML
# -------------------------
HTML_FORM = """
<!doctype html>
<title>Pickleball Fuel Finder (AI)</title>

<h1>🏓 AI Pickleball Meal Recommendations</h1>

<form method="post">

  <label>Height (cm):</label><br>
  <input type="number" name="height" required><br><br>

  <label>Weight (kg):</label><br>
  <input type="number" name="weight" required><br><br>

  <label>How long are you playing?</label><br>
  <select name="duration">
    <option value="short">Less than 1 hour</option>
    <option value="medium">1–3 hours</option>
    <option value="long">Tournament / 3+ hours</option>
  </select><br><br>

  <label>When do you want food for?</label><br>
  <select name="timing">
    <option value="pre">Before playing</option>
    <option value="during">Between games</option>
    <option value="post">After playing</option>
  </select><br><br>

  <input type="submit" value="Get AI Recommendations">

</form>

{% if daily_cal %}
<hr>
<h3>Estimated Daily Calories: {{ daily_cal }} kcal</h3>
{% endif %}

{% if meals %}
<h2>AI-Recommended Meals:</h2>
<ul>
  {% for meal in meals %}
    <li>
      <b>{{ meal.name }}</b> — {{ meal.calories }} kcal<br>
      <i>{{ meal.description }}</i>
    </li><br>
  {% endfor %}
</ul>
{% endif %}
"""

# -------------------------
# Main route
# -------------------------
@app.route("/", methods=["GET", "POST"])
def home():
    meals = None
    daily_cal = None

    if request.method == "POST":
        height = float(request.form.get("height"))
        weight = float(request.form.get("weight"))
        duration = request.form.get("duration")
        timing = request.form.get("timing")

        daily_cal = estimate_daily_calories(weight, height)

        profile = {
            "height": height,
            "weight": weight,
            "daily_cal": daily_cal,
            "duration": duration,
            "timing": timing,
        }

        meals = get_ai_meal_recommendations(profile)

    return render_template_string(
        HTML_FORM, meals=meals, daily_cal=daily_cal
    )


if __name__ == "__main__":
    app.run(debug=True)